# Generate simulated conversation data with your chat app or LLM endpoint

## Objective


Use the Simulator to simulate interactions with your local flow or LLM connection to generate target dataset for evaluation.

This tutorial uses the following Azure AI services:

- Access to Azure OpenAI Service - you can apply for access [here](https://go.microsoft.com/fwlink/?linkid=2222006)
- An Azure AI Studio project - go to [aka.ms/azureaistudio](https://aka.ms/azureaistudio) to create a project

## Time

You should expect to spend 10-15 minutes running this sample. 

## About this example

Large language models (LLMs) are known for their few-shot and zero-shot learning abilities, allowing them to function with minimal data. However, this limited data availability impedes thorough evaluation and optimization when you may not have test datasets to evaluate the quality and effectiveness of your generative AI application. Using GPT to simulate a user interaction with your application, with configurable tone, task and characteristics can help with stress testing your application under various environments, effectively gauging how a model responds to different inputs and scenarios.

There are two main scenarios for generating a simulated interaction (such as as conversation with a chat bot):

- Instance level with manual testing: generate one conversation at a time by manually inputting the task perameters such as name, profile, tone and task and iteratively tweaking it to see different outcomes for the simulated interaction.
- Bulk testing and evaluation orchestration: generate multiple interaction data samples (~100) at one time for a list of tasks or profiles to create an target dataset to evaluate your generative aI applications and streamline the data gathering/prep process.

This sample will be useful to developers who need a target test dataset generated that simulates a user persona interacting with your chat app or flow.



## Before you begin



### Installation

Install the following packages in the `requirements.txt` file required to execute this notebook. 



In [ ]:
# Install the packages
%pip install -r requirements.txt

### Parameters


Lets initialize some variables. For `subscription_id`, `resource_group_name` and `project_name`, you can go to the Project Overview page in the AI Studio. Replace the items in <> with values for your project. 

In [ ]:
# project details
subscription_id: str = "<your-subscription-id>"
resource_group_name: str = "<your-resource-group>"
project_name: str = "<your-project-name>"

should_cleanup: bool = False

## Connect to your project

To start with let us create a config file with your project details. This file can be used in this sample or other samples to connect to your workspace. 

In [ ]:
import json
from pathlib import Path

# Import packages
from azure.ai.resources.client import AIClient
from azure.ai.resources.entities import AzureOpenAIModelConfiguration
from azure.identity import DefaultAzureCredential
from azure.ai.generative.synthetic.simulator import Simulator, SimulatorTemplates

config = {
    "subscription_id": subscription_id,
    "resource_group": resource_group_name,
    "project_name": project_name,
}

p = Path("config.json")

with p.open(mode="w") as file:
    file.write(json.dumps(config))

Let us connect to the project

In [ ]:
# connects to project defined in the first config.json found in this or parent folders
ai_client = AIClient.from_config(DefaultAzureCredential())

## Retrieve Azure OpenAI details
We will use an Azure Open AI service to access the LLM. Let us get the details of these from your project.

In [ ]:
# Get the default Azure Open AI connection for your project
default_aoai_connection = ai_client.get_default_aoai_connection()
default_aoai_connection.set_current_environment()

## Define system connection and user connection for large language models (LLM)

The Simulator can simulate interactions between your large language model (i.e. "agent bot") and a system large language model (i.e. "human bot"). First we start by defining the connection to these two large language models to simulate an interaction. In this example we are simulating a conversation. For the human bot simulating the conversation with your agent bot, we recommend using Azure OpenAI GPT-4 model for the best outcomes. 

In [ ]:
# For simplicity, we define both the system bot and agent bot using the same LLM model.
system_bot_model = AzureOpenAIModelConfiguration.from_connection(
    connection=default_aoai_connection,
    model_name="gpt-35-turbo",
    deployment_name="gpt-35-turbo",
    max_tokens=500,
    temperature=0.0,
)

agent_bot_model = AzureOpenAIModelConfiguration.from_connection(
    connection=default_aoai_connection,
    model_name="gpt-35-turbo",
    deployment_name="gpt-35-turbo",
    max_tokens=500,
    temperature=0.0,
)

# Review template and define parameters

The human bot can be customized via parameters in a template. Parameters such as profile and tone (all required parameters) will be used to simulate the conversation with your agent bot. Provide a task to the human bot that will guide it in simulating the proper responses to your agent bot. The number of tasks will define the number of complete conversations that the simulator will simulate. The metadata (optional parameter which can either be a dictionary of key value pairs or a string text) will provide context to your agent bot on the human bot to answer appropriately.

In [ ]:
st = SimulatorTemplates()

# retrieve template for conversation task
conv_template = st.get_template("conversation")

# retrieve parameters in the template
conv_parameters = st.get_template_parameters("conversation")

# show the parameters in a template
print(conv_parameters)

Then we fill in the template parameters with our own simulated persona and task as well as their tone and additional metadata to inform the conversation as she is chatting with the `ChatBot`.

In [ ]:
# initialize the conversation template parameters
conv_parameters = {
    "name": "Jane",
    "profile": """
    Jane Doe is a 28-year-old outdoor enthusiast 
    who lives in Seattle, Washington. 
    She has a passion for exploring nature and loves going on camping and hiking trips with her friends. 
    She has recently become a member of the company's loyalty program and has achieved Bronze level status."""
    """Jane has a busy schedule, but she always makes time for her outdoor adventures.
    She is constantly looking for high-quality gear that can help her make the most of her trips and ensure she has a comfortable experience in the outdoors."""
    """Recently, Jane purchased a TrailMaster X4 Tent from the company. 
    This tent is perfect for her needs, as it is both durable and spacious, allowing her to enjoy her camping trips with ease. 
    The price of the tent was $250, and it has already proved to be a great investment."""
    "In addition to the tent, Jane also bought a Pathfinder Pro-1 Adventure Compass for $39.99. This compass has helped her navigate challenging trails with confidence, ensuring that she never loses her way during her adventures."
    "Finally, Jane decided to upgrade her sleeping gear by purchasing a CozyNights Sleeping Bag for $100. This sleeping bag has made her camping nights even more enjoyable, as it provides her with the warmth and comfort she needs after a long day of hiking.",
    "tone": "happy",
    "metadata": {
        "customer_info": "## customer_info      name: Jane Doe    age: 28     phone_number: 555-987-6543     email: jane.doe@example.com     address: 789 Broadway St, Seattle, WA 98101      loyalty_program: True     loyalty_program Level: Bronze        ## recent_purchases      order_number: 5  date: 2023-05-01  item: - description:  TrailMaster X4 Tent, quantity 1, price $250    item_number: 1   order_number: 18  date: 2023-05-04  item: - description:  Pathfinder Pro-1 Adventure Compass, quantity 1, price $39.99    item_number: 4   order_number: 28  date: 2023-04-15  item: - description:  CozyNights Sleeping Bag, quantity 1, price $100    item_number: 7"
    },
    "task": "Jane is trying to accomplish the task of finding out the best hiking backpacks suitable for her weekend camping trips, and how they compare with other options available in the market. She wants to make an informed decision before making a purchase from the outdoor gear company's website or visiting their physical store."
    "Jane uses Google to search for 'best hiking backpacks for weekend trips,' hoping to find reliable and updated information from official sources or trusted websites. She expects to see a list of top-rated backpacks, their features, capacity, comfort, durability, and prices. She is also interested in customer reviews to understand the pros and cons of each backpack."
    "Furthermore, Jane wants to see the specifications, materials used, waterproof capabilities, and available colors for each backpack. She also wants to compare the chosen backpacks with other popular brands like Osprey, Deuter, or Gregory. Jane plans to spend about 20 minutes on this task and shortlist two or three options that suit her requirements and budget."
    "Finally, as a Bronze level member of the outdoor gear company's loyalty program, Jane might also want to contact customer service to inquire about any special deals or discounts available on her shortlisted backpacks, ensuring she gets the best value for her purchase.",
    "chatbot_name": "ChatBot",
}

# Initialize the simulator with system and agent bot

In [ ]:
simulator = Simulator(systemConnection=system_bot_model, userConnection=agent_bot_model)

## Using asynchronous API to generate a conversation

In [ ]:
simulator = Simulator(systemConnection=system_bot_model, userConnection=agent_bot_model)

## Using synchronous API to generate a conversation

In [ ]:
conv = simulator.simulate(
    conv_template,
    conv_parameters,
    max_conversation_turns=6,
    api_call_delay_sec=10,
    api_call_retry_sleep_sec=10,
    api_call_retry_max_count=3,
)

print(conv)

# Initialize simulator with callback function

A callback function can be defined which is called when it is the agent bot's turn to generate a response. You can also pass in Conversation history and metadata as parameters in the `sim_callback()` function. In this example we are wrapping a local flow chatbot.

In [ ]:
async def sim_callback(question: str) -> dict:
    from promptflow import PFClient

    pf_client = PFClient()

    inputs = {"question": question}
    return pf_client.test(flow="./my_chatbot", inputs=inputs)["answer"]

In [ ]:
simulator = Simulator(simulate_callback=sim_callback, systemConnection=system_bot_model)

## Cleaning up

To clean up all Azure ML resources used in this example, you can delete the individual resources you created in this tutorial.

If you made a resource group specifically to run this example, you could instead [delete the resource group](https://learn.microsoft.com/en-us/azure/azure-resource-manager/management/delete-resource-group).

In [ ]:
if should_cleanup:
    # add clean up steps if needed
    pass